In [ ]:
# Upload the kaggle.json file
from google.colab import files
files.upload()


Saving kaggle (3).json to kaggle (3).json


{'kaggle (3).json': b'{"username":"rebecasujia","key":"470ec16247acca86eb9b405ebd08d4c6"}'}

In [ ]:
# Create the .kaggle directory
!mkdir -p ~/.kaggle

# Move the kaggle.json file to the .kaggle directory
!mv kaggle.json ~/.kaggle/

# Change the permissions of the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json


mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!ls ~/.kaggle


In [ ]:
!kaggle datasets download -d karakaggle/kaggle-cat-vs-dog-dataset

Dataset URL: https://www.kaggle.com/datasets/karakaggle/kaggle-cat-vs-dog-dataset
License(s): unknown
100% 787M/787M [00:31<00:00, 26.3MB/s]
100% 787M/787M [00:31<00:00, 25.8MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/kaggle-cat-vs-dog-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import zipfile
import os

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [17]:
train_dir = 'data/train'
test_dir = 'data/test'

In [18]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

In [30]:
with zipfile.ZipFile('/content/kaggle-cat-vs-dog-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/') # Extract to the current directory instead

In [31]:
train_dir = '/content/dogs-vs-cats/train'
validation_dir = '/content/dogs-vs-cats/validation'


In [32]:
train_dir = '/content/dogs-vs-cats/train'
validation_dir = '/content/dogs-vs-cats/validation'


In [42]:
import zipfile
import os

# Verify extraction path and directory existence
print(os.listdir('/content/'))  # List files and directories in '/content/'

# If the directory is missing, re-extract the zip file
with zipfile.ZipFile('/content/kaggle-cat-vs-dog-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

# Update paths if necessary - the extracted directory name is likely 'kagglecatsanddogs_3367a'
train_dir = '/content/data/train/cats'  # Adjust if the extracted directory is different
validation_dir = '/content/sample_data/test'  # Adjust if the extracted directory is different


['.config', 'kaggle-cat-vs-dog-dataset.zip', 'sample_data', 'kagglecatsanddogs_3367a', 'kaggle (3).json', 'data']


In [74]:
print('Cats:', os.listdir(cats_dir)[:5])  # Print first 5 cat images
print('Dogs:', os.listdir(dogs_dir)[:5])  # Print first 5 dog images

Cats: []
Dogs: []


In [46]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [76]:
class_names = train_generator.class_indices  # Get class names from the generator
print(class_names)

{}


In [55]:
import tensorflow as tf

def process(image,label):
    image = tf.image.resize(image,(150,150))
    return image,label

# Create new datasets from the generators
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=((150, 150, 3), (1,))  # Assuming images are RGB
)

validation_dataset = tf.data.Dataset.from_generator(
    lambda: validation_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=((150, 150, 3), (1,))
)

# Now you can use .map()
train_dataset = train_dataset.map(process)
validation_dataset = validation_dataset.map(process)

In [80]:
import os

def verify_image_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            try:
                with open(os.path.join(directory, filename), 'rb') as file:
                    file.read()
            except Exception as e:
                print(f"Error reading {filename}: {e}")

verify_image_files('data/train/cats')
verify_image_files('data/train/dogs')


In [81]:
from PIL import Image

def check_images(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            try:
                with Image.open(os.path.join(directory, filename)) as img:
                    img = img.resize((150, 150))
                    img_array = tf.keras.preprocessing.image.img_to_array(img)
                    if img_array.shape != (150, 150, 3):
                        print(f"Image {filename} has invalid shape: {img_array.shape}")
            except Exception as e:
                print(f"Error processing {filename}: {e}")

check_images('data/train/cats')
check_images('data/train/dogs')


In [83]:
import os

def check_directories(base_dir):
    if not os.path.exists(base_dir):
        print(f"Directory {base_dir} does not exist.")
    else:
        sub_dirs = os.listdir(base_dir)
        print(f"Found directories: {sub_dirs}")
        for sub_dir in sub_dirs:
            sub_dir_path = os.path.join(base_dir, sub_dir)
            if os.path.isdir(sub_dir_path):
                files = os.listdir(sub_dir_path)
                print(f"Files in {sub_dir}: {len(files)}")
                if len(files) == 0:
                    print(f"No files found in {sub_dir_path}")
            else:
                print(f"{sub_dir_path} is not a directory")

check_directories('data/train')


Found directories: ['cats', 'dogs']
Files in cats: 0
No files found in data/train/cats
Files in dogs: 0
No files found in data/train/dogs


In [85]:
import shutil

source_dir = 'data/extracted_images/train'  # Adjust if needed
target_dir = 'data/train'
cats_dir = os.path.join(target_dir, 'cats')
dogs_dir = os.path.join(target_dir, 'dogs')

# Verify existing files
def move_files(source_dir, target_dir, category):
    target_category_dir = os.path.join(target_dir, category)
    if not os.path.exists(target_category_dir):
        os.makedirs(target_category_dir)
    for filename in os.listdir(source_dir):
        if category in filename.lower():
            shutil.move(os.path.join(source_dir, filename), os.path.join(target_category_dir, filename))

move_files(source_dir, target_dir, 'cat')
move_files(source_dir, target_dir, 'dog')


FileNotFoundError: [Errno 2] No such file or directory: 'data/extracted_images/train'

In [77]:
# Preprocess the data
AUTOTUNE = tf.data.AUTOTUNE

def preprocess(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [150, 150])
    return image, label

train_dataset = train_dataset.map(preprocess, num_parallel_calls=AUTOTUNE)
validation_dataset = validation_dataset.map(preprocess, num_parallel_calls=AUTOTUNE)

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)


In [78]:
from tensorflow.keras import layers, models

# Build the model
model = models.Sequential([
    layers.InputLayer(input_shape=(150, 150, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
TypeError: `generator` yielded an element of shape (0, 150, 150, 3) where an element of shape (150, 150, 3) was expected.
Traceback (most recent call last):

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (0, 150, 150, 3) where an element of shape (150, 150, 3) was expected.


	 [[{{node PyFunc}}]]
	 [[IteratorGetNext]] [Op:__inference_one_step_on_iterator_7975]

In [57]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [58]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 15, 15, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │       3,211,776 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,453,121 (13.17 MB)

 Trainable params: 3,453,121 (13.17 MB)

 Non-trainable params: 0 (0.00 B)

In [59]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])